# E9. Recommender System
## 1. 데이터 및 라이브러리 불러오기

In [145]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [146]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [147]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [148]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [149]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [150]:
ratings = pd.merge(ratings, movies)
ratings

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [151]:
# 분석 1 - ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [152]:
# 분석 2 - rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [153]:
# 분석 3 - 가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [154]:
# 유저별 영화 시청 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [156]:
# 좋아하는 영화에 대한 id와 title list
my_favorite = [2019, 2000, 2102, 2297, 2022]

my_favorite_title = []
for i in my_favorite:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'woongcheol'의 user_id가 위 영화 5편을 5번 시청한 것으로 가정
my_movielist = pd.DataFrame({'user_id': ['woongcheol']*5, 'movie_id': my_favorite, 'count': [4]*5, 'title': my_favorite_title})

if not ratings.isin({'user_id':['woongcheol']})['user_id'].any():  # user_id 데이터 중복여부 확인
    ratings = ratings.append(my_movielist) 

ratings.tail(10)

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,woongcheol,2019,4,NaN,Seven Samurai (The Magnificent Seven) (Shichin...,NaN
1,woongcheol,2000,4,NaN,Lethal Weapon (1987),NaN
2,woongcheol,2102,4,NaN,Steamboat Willie (1940),NaN
3,woongcheol,2297,4,NaN,What Dreams May Come (1998),NaN
4,woongcheol,2022,4,NaN,"Last Temptation of Christ, The (1988)",NaN


In [157]:
# 사용하는 컬럼만 분류
using_cols = ['user_id', 'count', 'movie_id', 'title']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,count,movie_id,title
0,1,5,1193,One Flew Over the Cuckoo's Nest (1975)
1,2,5,1193,One Flew Over the Cuckoo's Nest (1975)
2,12,4,1193,One Flew Over the Cuckoo's Nest (1975)
3,15,4,1193,One Flew Over the Cuckoo's Nest (1975)
4,17,5,1193,One Flew Over the Cuckoo's Nest (1975)
5,18,4,1193,One Flew Over the Cuckoo's Nest (1975)
6,19,5,1193,One Flew Over the Cuckoo's Nest (1975)
7,24,5,1193,One Flew Over the Cuckoo's Nest (1975)
8,28,3,1193,One Flew Over the Cuckoo's Nest (1975)
9,33,5,1193,One Flew Over the Cuckoo's Nest (1975)


In [158]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [160]:
# 인덱싱 확인
print(user_to_idx['woongcheol'])
print(movie_to_idx['Terminator, The (1984)'])

6039
200


In [166]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing Fail!!
title column indexing OK!!


,user_id,count,movie_id,title
0,0,5,1193,0
1,1,5,1193,0
2,2,4,1193,0
3,3,4,1193,0
4,4,5,1193,0
...,...,...,...,...
0,6039,4,2019,980
1,6039,4,2000,648
2,6039,4,2102,2077
3,6039,4,2297,418


## 2. Matrix Factorization(MF)

In [167]:
num_user = ratings['user_id'].nunique()
num_user

6040

In [177]:
num_artist = ratings['title'].nunique()
num_artist

3628

In [169]:
ratings['count']

0    5
1    5
2    4
3    4
4    5
    ..
0    4
1    4
2    4
3    4
4    4
Name: count, Length: 836483, dtype: int64

In [170]:
ratings.user_id

0       0
1       1
2       2
3       3
4       4
     ... 
0    6039
1    6039
2    6039
3    6039
4    6039
Name: user_id, Length: 836483, dtype: int64

In [175]:
ratings.movie_id

0    1193
1    1193
2    1193
3    1193
4    1193
     ... 
0    2019
1    2000
2    2102
3    2297
4    2022
Name: movie_id, Length: 836483, dtype: int64

In [174]:
ratings.title

0       0
1       0
2       0
3       0
4       0
     ... 
0     980
1     648
2    2077
3     418
4    2446
Name: title, Length: 836483, dtype: int64

In [178]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_ratings = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_ratings

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [179]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [180]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [182]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [183]:
# 모델 훈련
als_model.fit(csr_ratings_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [187]:
woongcheol, terminator = user_to_idx['woongcheol'], movie_to_idx['Terminator, The (1984)']
woongcheol_vector, terminator_vector = als_model.user_factors[woongcheol], als_model.item_factors[terminator]

슝=3


In [189]:
woongcheol_vector

array([ 0.35026455, -0.3190053 , -0.5983445 , -0.21973148,  0.26837584,
       -0.10682967,  0.04717714,  0.02873887,  0.69866985, -0.12273936,
        0.1267442 ,  0.29216674,  0.5496587 , -0.14375447, -0.03281487,
        0.10400056,  0.34691998, -0.12998736, -0.2294388 ,  0.5728138 ,
       -0.32197982,  0.26189533,  0.23729268,  0.43449807, -0.22710481,
        0.31064674, -0.10183171,  0.11609445, -0.24848182, -0.41301435,
       -0.14918657, -0.26171327, -0.1294495 ,  0.22974005,  0.22923268,
       -0.30372402,  0.03406692, -0.40715525, -0.33867905,  0.13261135,
       -0.5639296 , -0.61113006, -0.00742833,  0.0224966 ,  0.34337994,
        0.5096111 ,  0.25439832,  0.29059774,  0.15851445,  0.3061198 ,
       -0.82054305,  0.07724702,  0.2538161 ,  0.64450556, -0.04928048,
        0.03430412,  0.1856681 , -0.14029591,  0.03165647, -0.17136951,
        0.00734881,  0.01695641, -0.07391451, -0.6025208 ,  0.6854726 ,
        0.09923339, -0.55569094, -0.29801908, -0.31502962,  0.85

In [190]:
terminator_vector

array([ 3.2909337e-02,  1.6254061e-03,  1.3173681e-02,  2.5205646e-02,
        3.8252209e-04, -1.9410746e-02,  2.0263880e-02, -4.4035795e-03,
        9.0996455e-03, -3.3662629e-05,  1.7821760e-03, -1.5229891e-03,
        2.6437700e-02,  2.7616846e-04,  9.3243057e-03,  6.2366659e-03,
       -2.1948079e-02,  1.4390189e-02, -2.4704114e-03,  6.3805715e-03,
       -8.8022929e-04,  5.4072342e-03,  1.0223408e-02,  3.2565542e-02,
        2.5395434e-03,  1.1738979e-02,  2.2587933e-02,  9.7922795e-03,
        2.3200221e-03,  3.1356536e-02,  3.8324082e-03, -1.7277419e-03,
        4.4545168e-03,  1.3674503e-02, -6.3964315e-03, -2.1085089e-02,
        2.3893543e-02,  6.9570786e-04,  4.4620387e-02, -1.1885443e-02,
        1.1955500e-02,  1.2717867e-02,  2.7324269e-03,  2.7029952e-02,
       -3.5864566e-03, -1.2127888e-02,  2.8786924e-02,  2.9572219e-02,
        3.1778738e-03,  6.3126013e-03, -1.6651473e-03, -5.6767180e-03,
        7.3378105e-03,  4.3687723e-03,  1.8406062e-03,  1.0937613e-03,
      

In [191]:
# woongcheol과 terminator를 내적하는 코드(선호 영화)
np.dot(woongcheol_vector, terminator_vector)

0.0033373556

In [192]:
# woongcheol과 indian을 내적하는 코드(비선호 영화)
indian = movie_to_idx['One Little Indian (1973)']
indian_vector = als_model.item_factors[indian]
np.dot(woongcheol_vector, indian_vector)

0.0018208637

In [193]:
favorite_movie = 'Terminator, The (1984)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(200, 0.99999994),
 (651, 0.77996826),
 (194, 0.7189378),
 (865, 0.6886521),
 (193, 0.6788677),
 (92, 0.61119175),
 (124, 0.59581405),
 (680, 0.51780003),
 (928, 0.51611483),
 (117, 0.516093),
 (172, 0.50167316),
 (62, 0.49988487),
 (120, 0.47719738),
 (44, 0.47408146),
 (882, 0.4573325)]

In [195]:
#artist_to_idx 변환 ->index로부터 artist 이름을 얻는 dict를 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Terminator, The (1984)',
 'Aliens (1986)',
 'Die Hard (1988)',
 'Predator (1987)',
 'Alien (1979)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Blade Runner (1982)',
 'Robocop (1987)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Indiana Jones and the Last Crusade (1989)',
 'Total Recall (1990)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)']

In [196]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [197]:
get_similar_movie('Mad Max 2 (a.k.a. The Road Warrior) (1981)')

['Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Mad Max (1979)',
 'Mad Max Beyond Thunderdome (1985)',
 'Running Man, The (1987)',
 'Big Trouble in Little China (1986)',
 'Predator (1987)',
 'Terminator, The (1984)',
 'Aliens (1986)',
 'Starman (1984)',
 'Escape from New York (1981)']

In [198]:
get_similar_movie('Starman (1984)')

['Starman (1984)',
 'Alien Nation (1988)',
 'Dreamscape (1984)',
 'Fabulous Baker Boys, The (1989)',
 'Running Man, The (1987)',
 'Mad Max Beyond Thunderdome (1985)',
 "Clara's Heart (1988)",
 'Cocoon (1985)',
 "Prizzi's Honor (1985)",
 'Mad Max (1979)']

In [203]:
user = user_to_idx['woongcheol']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
movie_recommended

[(57, 0.18457669),
 (692, 0.18072756),
 (198, 0.17629653),
 (650, 0.17617317),
 (380, 0.16545533),
 (273, 0.16516545),
 (607, 0.16065538),
 (120, 0.15766189),
 (183, 0.15308097),
 (527, 0.14807473),
 (6, 0.14528234),
 (70, 0.14264911),
 (1106, 0.13998802),
 (172, 0.13957915),
 (325, 0.13413773),
 (601, 0.12587434),
 (87, 0.12497087),
 (318, 0.124057196),
 (326, 0.12127824),
 (192, 0.12083492)]

In [204]:
# 인덱스 변환 - 영화제목
[idx_to_movie[i[0]] for i in movie_recommended]

['Untouchables, The (1987)',
 'Blues Brothers, The (1980)',
 'Good, The Bad and The Ugly, The (1966)',
 'Lethal Weapon 2 (1989)',
 'Godfather: Part II, The (1974)',
 'Fight Club (1999)',
 'Godfather, The (1972)',
 'Raiders of the Lost Ark (1981)',
 'Butch Cassidy and the Sundance Kid (1969)',
 'Batman (1989)',
 'Ben-Hur (1959)',
 'Ran (1985)',
 'Midnight Run (1988)',
 'Indiana Jones and the Last Crusade (1989)',
 'Romancing the Stone (1984)',
 'Citizen Kane (1941)',
 'Braveheart (1995)',
 'Rosencrantz and Guildenstern Are Dead (1990)',
 'Top Gun (1986)',
 'Fistful of Dollars, A (1964)']

In [208]:
# 유사도가 높은 영화목록
contribution = movie_to_idx['Top Gun (1986)']
explain = als_model.explain(user, csr_ratings, itemid=contribution)
                            
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Lethal Weapon (1987)', 0.05341556495554612),
 ('What Dreams May Come (1998)', 0.025423022801697445),
 ('Last Temptation of Christ, The (1988)', 0.022817431222523533),
 ('Steamboat Willie (1940)', 0.015651564544198894),
 ('Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)',
  0.000302225271756053)]